In [21]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..','..','..')))

from pudl import init
from pudl import constants as pc
import pudl.models.glue, pudl.models.eia860, pudl.models.eia923
import pudl.models.ferc1
import pudl.models.epacems
import pudl.extract.eia923
import pudl.transform.eia
import pudl.output.pudltabl
import pudl.helpers
import pudl.datastore
import pudl.output
import pudl.analysis.analysis

pudl_engine = init.connect_db(testing=False)
f1_engine = pudl.extract.ferc1.connect_db()
plt.style.use('ggplot')
mpl.rcParams['figure.figsize'] = (10,6)
mpl.rcParams['figure.dpi'] = 150

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
import sqlalchemy as sa

In [53]:
# Make sure we get an exhaustive list.
pudl.extract.ferc1.init_db(refyear=2017, years=range(2004,2018), def_db=True)

Start ferc mirror db ingest at Tuesday, 20. November 2018 11:35AM
Defining new FERC Form 1 DB based on 2017...
Clearing any existing FERC Form 1 database MetaData...
Ingesting FERC Form 1 Data from 2004...
Ingesting FERC Form 1 Data from 2005...
Ingesting FERC Form 1 Data from 2006...
Ingesting FERC Form 1 Data from 2007...
Ingesting FERC Form 1 Data from 2008...
Ingesting FERC Form 1 Data from 2009...
Ingesting FERC Form 1 Data from 2010...
Ingesting FERC Form 1 Data from 2011...
Ingesting FERC Form 1 Data from 2012...
Ingesting FERC Form 1 Data from 2013...
Ingesting FERC Form 1 Data from 2014...
Ingesting FERC Form 1 Data from 2015...
Ingesting FERC Form 1 Data from 2016...
Ingesting FERC Form 1 Data from 2017...


In [94]:
new_fercplants = pudl.analysis.analysis.fercplants(plant_tables=['f1_steam',
                                                                 'f1_gnrt_plant',
                                                                 'f1_hydro',
                                                                 'f1_pumped_storage'],
                                                   new=True, years=range(2004,2018), min_capacity=0.0)
pudl_fercutils = pd.read_sql('''select utility_id_ferc1, utility_name_ferc1 from utilities_ferc;''', pudl_engine)
new_fercutils = new_fercplants[['utility_id_ferc1','utility_name_ferc1']].drop_duplicates()
new_fercutils = new_fercutils.set_index(['utility_id_ferc1',])
pudl_fercutils = pudl_fercutils.set_index(['utility_id_ferc1',])
new_index = new_fercutils.index.difference(pudl_fercutils.index)
really_new_fercutils = new_fercutils.loc[new_index].reset_index()

Defining new FERC Form 1 DB based on 2017...
Clearing any existing FERC Form 1 database MetaData...


In [13]:
all_ferc_respondents = pd.read_sql('''select distinct respondent_id, respondent_name from f1_respondent_id order by respondent_id asc;''', f1_engine)

In [ ]:
all_ferc_respondents.to_excel('all_ferc_respondents_2004-2017.xlsx', index=False)

In [14]:
small_plants_2017 = pd.read_sql('''SELECT report_year, respondent_id, row_number, spplmnt_num, plant_name, kind_of_fuel \
                                        FROM f1_gnrt_plant \
                                        WHERE report_year=2017;''', f1_engine)
small_plants_2017['record_number'] = small_plants_2017['row_number'] + 46*small_plants_2017['spplmnt_num']
small_plants_2017 = small_plants_2017.drop(['row_number','spplmnt_num'],axis=1)
small_plants_2017 = small_plants_2017[['report_year','respondent_id','record_number','plant_name','kind_of_fuel']]
small_plants_2017.to_excel('small_plants_2017.xlsx', index=False)

In [95]:
new_fercplants.to_excel('ferc1_unmapped_plants_2017.xlsx')

In [ ]:
new_fercplants.to_csv('ferc1_unmapped_plants_2017.csv', sep=',')